## KAGGLE TITANIC



### 1. Data Cleaning

In [40]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [41]:
data_train=pd.read_csv("data/train.csv")
data_train=shuffle(data_train,random_state=3)
data_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
395,396,0,3,"Johansson, Mr. Erik",male,22.0,0,0,350052,7.7958,NaN,S
85,86,1,3,"Backstrom, Mrs. Karl Alfred (Maria Mathilda Gu...",female,33.0,3,0,3101278,15.8500,NaN,S
201,202,0,3,"Sage, Mr. Frederick",male,NaN,8,2,CA. 2343,69.5500,NaN,S
542,543,0,3,"Andersson, Miss. Sigrid Elisabeth",female,11.0,4,2,347082,31.2750,NaN,S
702,703,0,3,"Barbara, Miss. Saiide",female,18.0,0,1,2691,14.4542,NaN,C


In [42]:
cabin=data_train[['Cabin']]
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder=OrdinalEncoder()
cabin_encoded=ordinal_encoder.fit_transform(cabin)
ordinal_encoder.categories_

[array(['A10', 'A14', 'A16', 'A19', 'A20', 'A23', 'A24', 'A26', 'A31',
        'A32', 'A34', 'A36', 'A5', 'A6', 'A7', 'B101', 'B102', 'B18',
        'B19', 'B20', 'B22', 'B28', 'B3', 'B30', 'B35', 'B37', 'B38',
        'B39', 'B4', 'B41', 'B42', 'B49', 'B5', 'B50', 'B51 B53 B55',
        'B57 B59 B63 B66', 'B58 B60', 'B69', 'B71', 'B73', 'B77', 'B78',
        'B79', 'B80', 'B82 B84', 'B86', 'B94', 'B96 B98', 'C101', 'C103',
        'C104', 'C106', 'C110', 'C111', 'C118', 'C123', 'C124', 'C125',
        'C126', 'C128', 'C148', 'C2', 'C22 C26', 'C23 C25 C27', 'C30',
        'C32', 'C45', 'C46', 'C47', 'C49', 'C50', 'C52', 'C54', 'C62 C64',
        'C65', 'C68', 'C7', 'C70', 'C78', 'C82', 'C83', 'C85', 'C86',
        'C87', 'C90', 'C91', 'C92', 'C93', 'C95', 'C99', 'D', 'D10 D12',
        'D11', 'D15', 'D17', 'D19', 'D20', 'D21', 'D26', 'D28', 'D30',
        'D33', 'D35', 'D36', 'D37', 'D45', 'D46', 'D47', 'D48', 'D49',
        'D50', 'D56', 'D6', 'D7', 'D9', 'E10', 'E101', 'E12', 'E121',

In [43]:
data_train[['Cabin']]=cabin_encoded
train_labels=data_train["Survived"]
data_train.loc[data_train['Sex'] =='male','s'] = 1 
data_train.loc[data_train['Sex'] =='female','s'] = 0
data_train=data_train.drop(['PassengerId','Name','Ticket','Embarked','Sex','Survived'],axis=1)
data_train.to_numpy()
data_train=np.nan_to_num(data_train)
train_labels.to_numpy()
train_examples=data_train
X = train_examples
y = train_labels

X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=0.8)
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)
BATCH_SIZE = int((X_train.shape[0])/1000)

### TORCH IMPLEMENTATION BEGINS

In [44]:
import torch
import torch.nn as nn

In [45]:
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()
X_train= sc.fit_transform(X_train)
X_test= sc.fit_transform(X_test)
y_train=pd.Series.to_numpy(y_train)
y_test=pd.Series.to_numpy(y_test)

X_train=torch.from_numpy(X_train)
X_test=torch.from_numpy(X_test)
y_train=torch.from_numpy(y_train)
y_test=torch.from_numpy(y_test)
y_train = y_train.view(y_train.shape[0],1)
y_test = y_test.view(y_test.shape[0],1)

In [46]:
(n_samples,features)=X_train.shape
input_size=features
output_size=1

In [47]:
class training(nn.Module):

    def __init__(self,input):
        super(training,self).__init__()

        self.lin1=nn.Linear(input,input)
        self.lin2=nn.Linear(input,14)
        self.lin3=nn.Linear(14,7)
        self.lin4=nn.Linear(7,1)

    def forward(self,x):
        x=torch.relu(self.lin1(x))
        x=torch.relu(self.lin2(x))
        x=torch.relu(self.lin3(x))
        return torch.sigmoid(self.lin4(x))
    

In [48]:
model=training(input_size)

In [49]:
y_train.shape

torch.Size([712, 1])

In [50]:
learning_rate=0.01
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)
criterion = nn.BCELoss()
num_epochs=25000
for epochs in range(num_epochs):
    y_pred=model(X_train.float())
    loss=criterion(y_pred.float(),y_train.float())

    loss.backward()

    optimizer.step()

    optimizer.zero_grad()

    if(epochs%100 == 0):
        print(f'epoch {epochs+1}, loss {loss.item():.4f} ')


epoch 1, loss 0.6767 
epoch 101, loss 0.6701 
epoch 201, loss 0.6671 
epoch 301, loss 0.6651 
epoch 401, loss 0.6633 
epoch 501, loss 0.6613 
epoch 601, loss 0.6591 
epoch 701, loss 0.6566 
epoch 801, loss 0.6534 
epoch 901, loss 0.6495 
epoch 1001, loss 0.6443 
epoch 1101, loss 0.6374 
epoch 1201, loss 0.6282 
epoch 1301, loss 0.6156 
epoch 1401, loss 0.5984 
epoch 1501, loss 0.5755 
epoch 1601, loss 0.5467 
epoch 1701, loss 0.5152 
epoch 1801, loss 0.4871 
epoch 1901, loss 0.4672 
epoch 2001, loss 0.4552 
epoch 2101, loss 0.4479 
epoch 2201, loss 0.4434 
epoch 2301, loss 0.4403 
epoch 2401, loss 0.4381 
epoch 2501, loss 0.4365 
epoch 2601, loss 0.4352 
epoch 2701, loss 0.4341 
epoch 2801, loss 0.4332 
epoch 2901, loss 0.4325 
epoch 3001, loss 0.4319 
epoch 3101, loss 0.4313 
epoch 3201, loss 0.4307 
epoch 3301, loss 0.4302 
epoch 3401, loss 0.4298 
epoch 3501, loss 0.4293 
epoch 3601, loss 0.4289 
epoch 3701, loss 0.4285 
epoch 3801, loss 0.4281 
epoch 3901, loss 0.4278 
epoch 4001, 

In [51]:
with torch.no_grad():
    y_predicted=model(X_test.float())
    y_predicted_cls= y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'Accuracy : {acc:.3f} ')

Accuracy : 0.800 


### 3. CSV SUBMISSION

In [52]:
test=pd.read_csv("data/test.csv")
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [53]:
test=pd.read_csv("data/test.csv")
test_id=test["PassengerId"]
test.loc[test['Sex'] =='male','s'] = 1 
test.loc[test['Sex'] =='female','s'] = 0
cabin=test[['Cabin']]
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder=OrdinalEncoder()
cabin_encoded=ordinal_encoder.fit_transform(cabin)
test[['Cabin']]=cabin_encoded
test=test.drop(['Name','Ticket','Embarked','Sex','PassengerId'],axis=1)
test.head()


,Pclass,Age,SibSp,Parch,Fare,Cabin,s
0,3,34.5,0,0,7.8292,NaN,1.0
1,3,47.0,1,0,7.0000,NaN,0.0
2,2,62.0,0,0,9.6875,NaN,1.0
3,3,27.0,0,0,8.6625,NaN,1.0
4,3,22.0,1,1,12.2875,NaN,0.0


In [54]:
test.to_numpy()
test=np.nan_to_num(test)
test=torch.from_numpy(test)
with torch.no_grad():
    ans=model(test.float())
    ans= ans.round()    


In [55]:
data=pd.DataFrame()
data.insert(0,"PassengerId",test_id)
data.insert(1,"Survived",ans.int())
data.to_csv("data/model1_titanic.csv",index=False)